In [ ]:
import os
from typing import Callable, Dict, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import tensorflow as tf

In [ ]:
os.chdir("/home/ec2-user/cs231n")
os.listdir()

In [ ]:
MAX_EPOCHS = 400
POS_DIR = "data/pos_d1"
NEG_DIR = "data/neg_d1"

# Load data

In [ ]:
def load_data(
    dir: str,
    target_shape=[224, 224]
  ) -> np.ndarray:
  """Load data.

  Args
  ----
    dir: Directory from which to load images.
    target_shape: Shape for output

  """
  orig_dir = os.getcwd()
  os.chdir(dir)
  files = os.listdir()
  arrays = []
  for in_file in files:
    arrays.append(np.load(in_file, allow_pickle=True))
  out = np.stack(arrays)
  if target_shape != [256, 256]:
    out = tf.image.resize(out, target_shape)
    out = out.numpy()
  os.chdir(orig_dir)
  return out

## Training validation split


In [ ]:
def gen_sampling_weights(y: np.ndarray) -> np.ndarray:
  """Inverse probability sampling weights.
  
  Prepares a weight vector with weight inversely proportional to the
  frequency of the class in the input vector.

  Args
  ----
    y: Binary [0, 1] labels.
  
  """
  n1 = sum(y == 1)
  n0 = sum(y == 0)
  p1 = n1 / (n1 + n0)
  p0 = 1.0 - p1
  w1 = 1.0 / p1
  w0 = 1.0 / p0
  out = np.ones_like(y)
  out[y == 1] = w1
  out[y == 0] = w0
  return out

In [ ]:
def prep_data_for_model(
    pos: np.ndarray,
    neg: np.ndarray,
    train_prop = 0.7,
) -> Dict[str, np.ndarray]:
  """Prepare data for modeling.

  Args
  ----
    pos: Positive example images.
    neg: Negative example images.
    train_prop: Proportion of data for training. Remaining is allocated to
    validation.
  
  """
  n1 = pos.shape[0]
  n0 = neg.shape[0]
  cut1 = int(train_prop * n1)
  cut0 = int(train_prop * n0)

  # Split into training and validation sets.
  pos_train = pos[:cut1, :]
  pos_val = pos[cut1:, :]
  neg_train = neg[:cut0, :]
  neg_val = neg[cut0:, :]
  x_train = np.concatenate((pos_train, neg_train), axis=0)
  x_val = np.concatenate((pos_val, neg_val), axis=0)

  # Prepare labels.
  y1_train = np.ones(pos_train.shape[0])
  y0_train = np.zeros(neg_train.shape[0])
  y_train = np.concatenate((y1_train, y0_train), axis=0)

  y1_val = np.ones(pos_val.shape[0])
  y0_val = np.zeros(neg_val.shape[0])
  y_val = np.concatenate((y1_val, y0_val), axis=0)

  # Prepare weights.
  w_train = gen_sampling_weights(y_train)
  w_val = gen_sampling_weights(y_val)

  # Output.
  return {
      "x_train": x_train,
      "y_train": y_train,
      "w_train": w_train,
      "x_val": x_val,
      "y_val": y_val,
      "w_val": w_val
  }

# Model

## Generators

In [ ]:
def prep_generators(
    data: Dict[str, np.ndarray],
    train_batch=64,
    val_batch=32,
) -> Dict:
  """Prepare data generators

  Generators apply random relections and rotations.

  """
  transformer = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=360,
    horizontal_flip=True,
    vertical_flip=True,
  )

  train_generator = transformer.flow(
    x=data["x_train"],
    y=data["y_train"],
    batch_size=train_batch,
    sample_weight=data["w_train"]
  )

  val_generator = transformer.flow(
    x=data["x_val"],
    y=data["y_val"],
    batch_size=val_batch,
    sample_weight=data["w_val"]
  )

  return {
      "train_gen": train_generator,
      "val_gen": val_generator
  }


## Compile

In [ ]:
def _conv_layer(nodes, l2=0.0, name="conv") -> tf.keras.layers.Layer:
  return tf.keras.layers.Conv2D(
        filters=nodes,
        kernel_size=3,
        activation="relu",
        kernel_regularizer=tf.keras.regularizers.L2(l2),
        name=name,
        padding="same",
      )

In [ ]:
def _custom_model(
    conv_blocks=3,
    drop_prob=0.0,
    l2=0.0,
    min_nodes=32,
    max_nodes=256
) -> tf.keras.Model:
  """Architecture of custom model."""
  # First convolutional block.
  nodes = min_nodes
  model = tf.keras.Sequential([
    _conv_layer(nodes, l2=l2, name="1a"),
    _conv_layer(nodes, l2=l2, name="1b")                          
  ])
  # Dropout.
  if drop_prob > 0:
    model.add(tf.keras.layers.Dropout(drop_prob))

  # Subsequent convolutional blocks.
  for i in range(1, conv_blocks):
    nodes = min(nodes * 2, max_nodes)
    model.add(tf.keras.layers.MaxPool2D(name=f"max_pool{i}"))
    model.add(tf.keras.layers.BatchNormalization(name=f"batch_norm{i}"))
    model.add(_conv_layer(nodes, l2=l2, name=f"{i+1}a"))
    model.add(_conv_layer(nodes, l2=l2, name=f"{i+1}b"))
    if drop_prob > 0:
      model.add(tf.keras.layers.Dropout(drop_prob))

  # Pool and flatten.
  model.add(tf.keras.layers.GlobalAveragePooling2D(name="global_pool"))
  model.add(tf.keras.layers.Flatten(name="flat"))
  
  return model

In [ ]:
def _xception_model(
    input_shape: Tuple[int],
    unfreeze_first=False,
    unfreeze_last=False
) -> tf.keras.Model:
  """Prepare exception model."""
  model = tf.keras.applications.xception.Xception(
      include_top=False,
      input_shape=input_shape,
      pooling="avg"
    )
  model.trainable = False

  if unfreeze_first:
    model.layers[1].trainable = True
    model.layers[4].trainable = True

  if unfreeze_last:
    model.layers[126].trainable = True
    model.layers[129].trainable = True

  return model

In [ ]:
def define_model(
    custom_layers=3,
    drop_prob=0.0,
    embed="custom",
    extra_nodes=0,
    input_shape=(224, 224, 3),
    lr=1e-4,
    l2=0.0,
    unfreeze_first=False,
    unfreeze_last=False
) -> tf.keras.Model:
  """Define model.

  Args
  ----
    custom_layers: Layers for custom model.
    drop_prob: Dropout probability.
    embed: Embedding from ["custom", "mobile", "resnet", "xception"].
    extra_nodes: Nodes in extra layer between embedder and output.
    input_shape: Shape of inputs (excluding batch dimension).
    lr: Learning rate.
    l2: L2 weight decay penalty.
    unfreeze_first: Unfreeze first few layers of pretrained model?
    unfreeze_last: Unfreeze last few layers of pretrained model?
  
  """
  # Embedding module.
  if embed == "mobile":
    preprocessor = tf.keras.applications.mobilenet_v2.preprocess_input
    embedder = tf.keras.applications.mobilenet_v2.MobileNetV2(
      include_top=False,
      input_shape=input_shape,
      pooling="avg"
    )
    embedder.trainable = False
  # Resnet50.
  elif embed == "resnet":
    preprocessor = tf.keras.applications.resnet.preprocess_input
    embedder = tf.keras.applications.resnet50.ResNet50(
      include_top=False,
      input_shape=input_shape,
      pooling="avg"
    )
    embedder.trainable = False
  # Xception.
  elif embed == "xception":
    preprocessor = tf.keras.applications.xception.preprocess_input
    embedder = _xception_model(input_shape, unfreeze_first, unfreeze_last)
  # Basic (default).
  else:
    preprocessor = tf.identity
    embedder = _custom_model(
        conv_blocks=custom_layers, drop_prob=drop_prob, l2=l2)

  # Model.
  inputs = tf.keras.Input(shape=input_shape, name="input")
  h = preprocessor(inputs)
  h = embedder(h, training=False)
  if (extra_nodes > 0):
    h = tf.keras.layers.Dense(extra_nodes, activation="relu")(h)
  outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(h)
  model = tf.keras.Model(inputs, outputs)

  # Compile.
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[
          tf.keras.metrics.BinaryAccuracy(name="acc"),
          tf.keras.metrics.AUC(name="auc")
        ]
  )
  return model

# Training

In [ ]:
def train_and_eval(
    pos_dir=POS_DIR,
    neg_dir=NEG_DIR,
    custom_layers=3,
    drop_prob=0.0,
    embed="custom",
    extra_nodes=0,
    lr=1e-4,
    l2=0.0,
    max_epochs=MAX_EPOCHS,
    patience=10,
    unfreeze_first=False,
    unfreeze_last=False,
) -> Dict[str, float]:
  """Train and evaluate a model given the input data.
  
  Args
  ----
    pos_dir: Directory containing positive examples.
    neg_dir: Directory containing negative examples.
    custom_layers: Layers for custom model.
    drop_prob: Dropout probability.
    embed: Embedding from ["custom", "mobile", "resnet", "xception"].
    extra_nodes: Nodes in extra layer between embedder and output.
    lr: Learning rate.
    l2: L2 penalty.
    max_epochs: Maximum training epochs.
    patience: Patience for early stopping.
    unfreeze_first: Unfreeze first few layers of pretrained model?
    unfreeze_last: Unfreeze last few layers of pretrained model?
  
  """
  # Load data.
  pos = load_data(pos_dir)
  neg = load_data(neg_dir)
  data = prep_data_for_model(pos, neg)

  # Prepare model.
  model = define_model(
      custom_layers=custom_layers,
      drop_prob=drop_prob,
      embed=embed, 
      extra_nodes=extra_nodes,
      lr=lr,
      l2=l2,
      unfreeze_first=unfreeze_first,
      unfreeze_last=unfreeze_last
    )
  print(model.summary())

  # Prepare generators.
  gen = prep_generators(data)

  # Callbacks.
  callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir="final"),
    tf.keras.callbacks.EarlyStopping(
        patience=patience, restore_best_weights=True)
  ]

  # Train model.
  hist = model.fit(
    x=gen["train_gen"],
    validation_data=gen["val_gen"],
    validation_freq=10,
    epochs=max_epochs,
    callbacks=callbacks,
    verbose=0
  )

  # Evaluate model.
  train_eval = model.evaluate(
    x=data["x_train"],
    y=data["y_train"]
  )
  val_eval = model.evaluate(
    x=data["x_val"],
    y=data["y_val"]
  )

  # Output.
  return {
      "train_loss": train_eval[0],
      "train_acc":  train_eval[1],
      "train_auc":  train_eval[2],
      "val_loss": val_eval[0],
      "val_acc":  val_eval[1],
      "val_auc":  val_eval[2]
  }

# Evaluation

In [ ]:
def _fold_key(current_fold, length, total_folds) -> np.ndarray:
  """Key to select observations in current fold."""
  obs_per_fold = np.ceil(length / total_folds)
  idx0 = int((current_fold) * obs_per_fold)
  idx1 = int((current_fold + 1) * obs_per_fold)
  key = np.zeros((length,), dtype=bool)
  key[idx0:idx1] = True
  return key

In [ ]:
def model_eval(
    Model: Callable,
    pos_dir=POS_DIR,
    neg_dir=NEG_DIR,
    folds=5,
    max_epochs=MAX_EPOCHS,
) -> Dict:
  """Evaluate model performance via cross validation.
  
  Args
  ----
    model: *Function* that returns the model to train. 
      Should require no arguments.
    pos_dir: Directory containing positive examples.
    neg_dir: Directory containing negative examples.
    folds: Cross validation folds.
  
  """
  # Load data.
  pos = load_data(pos_dir)
  neg = load_data(neg_dir)

  n_pos = pos.shape[0]
  n_neg = neg.shape[0]

  # Model evaluation.
  evals = {"fold": [], "acc": [], "auc": []}

  # Model predictions.
  pos_yhat = np.zeros((n_pos, ))
  neg_yhat = np.zeros((n_neg, ))

  # Model embeddings.
  model = Model()
  embed_shape = model.layers[-2].output_shape[1]
  pos_embed = np.zeros((n_pos, embed_shape))
  neg_embed = np.zeros((n_neg, embed_shape))

  # Loop over folds.
  for k in range(folds):
    pos_key = _fold_key(k, n_pos, folds)
    neg_key = _fold_key(k, n_neg, folds)

    pos_eval = pos[pos_key, :]
    neg_eval = neg[neg_key, :]
    pos_train = pos[~pos_key, :]
    neg_train = neg[~neg_key, :]

    data = prep_data_for_model(pos_train, neg_train)
    
    # Train model.
    model = Model()
    gen = prep_generators(data)
    callbacks = [
      tf.keras.callbacks.EarlyStopping(
          patience=10, restore_best_weights=True)
    ]
    print(f"\n\nStarting training for fold {k}.\n\n")
    hist = model.fit(
      x=gen["train_gen"],
      validation_data=gen["val_gen"],
      validation_freq=10,
      epochs=max_epochs,
      callbacks=callbacks,
      verbose=0
    )

    # Evaluate model.
    x = np.concatenate((pos_eval, neg_eval), axis=0)
    y_pos = np.ones(pos_eval.shape[0])
    y_neg = np.zeros(neg_eval.shape[0])
    y = np.concatenate((y_pos, y_neg), axis=0)
    fold_eval = model.evaluate(x, y)
    
    evals["fold"].append(k)
    evals["acc"].append(fold_eval[1])
    evals["auc"].append(fold_eval[2])

    # Model predictions.
    pos_yhat[pos_key] = np.squeeze(model.predict(pos_eval))
    neg_yhat[neg_key] = np.squeeze(model.predict(neg_eval))

    # Model embeddings.
    embedder = tf.keras.Sequential(model.layers[:-1])
    pos_embed[pos_key] = embedder(pos_eval)
    neg_embed[neg_key] = embedder(neg_eval)
  
  # Output.
  return {
      "eval": pd.DataFrame(evals, index=evals["fold"]),
      "pos_yhat": pos_yhat,
      "neg_yhat": neg_yhat,
      "pos_embed": pos_embed,
      "neg_embed": neg_embed
  }

# Depth of max projection

In [ ]:
if False:
  depths = [1, 3, 5, 7] 

  results = {
    "depth": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for d in depths:
    results["depth"].append(d)
  
  print(f"\n\nStarting depth {d}.\n\n")
  out = train_and_eval(
    pos_dir=f"data/pos_d{d}",
    neg_dir=f"data/neg_d{d}",
  )

  for key in out:
    results[key].append(out[key])

  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/depth_experiment.tsv", sep="\t")

# Embedding Experiment
* Vary the model used to generate embeddings.

In [ ]:
if False:
  embedder = ["custom", "mobile", "resnet", "xception"]

  results = {
    "embedder": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for e in embedder:
    results["embedder"].append(e)
    
    print(f"\n\nStarting model with {e} embedding.\n\n")
    out = train_and_eval(
      embed=e,
    )

    for key in out:
      results[key].append(out[key])
  
  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/embed_experiment.tsv", sep="\t")

## Extra fully connected layer between embedder and output

In [ ]:
if False:
  embedder = ["custom", "mobile", "resnet", "xception"]

  results = {
    "embedder": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for e in embedder:
    results["embedder"].append(e)
    
    print(f"\n\nStarting model with {e} embedding.\n\n")
    out = train_and_eval(
      embed=e,
      extra_nodes=32
    )

    for key in out:
      results[key].append(out[key])

  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/embed_extra_layer_experiment.tsv", sep="\t")

## Xception fine-tuning

In [ ]:
if False:
  unfreeze_first = [False, False, True, True]
  unfreeze_last  = [False, True, False, True]

  results = {
    "unfreeze_first": [],
    "unfreeze_last": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for (uf, ul) in zip(unfreeze_first, unfreeze_last):
    results["unfreeze_first"].append(uf)
    results["unfreeze_last"].append(ul)
    
    print(f"""\n\n
            Starting model with first unfrozen ({uf})
            and last unfrozen ({ul}).\n\n""")
    out = train_and_eval(
      embed="xception",
      extra_nodes=32,
      unfreeze_first=uf,
      unfreeze_last=ul
    )

    for key in out:
      results[key].append(out[key])

  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/xception_fine_tuning_experiment.tsv", sep="\t")

## Xception extra layer nodes

In [ ]:
if False:
  nodes = [0, 32, 64, 128]

  results = {
    "nodes": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for n in nodes:
    results["nodes"].append(n)
    
    print(f"""\n\nStarting model with {n} nodes.\n\n""")
    out = train_and_eval(
      embed="xception",
      extra_nodes=n
    )

    for key in out:
      results[key].append(out[key])

  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/xception_extra_nodes_experiment.tsv", sep="\t")

# Custom Model Optimization

## Convolutional blocks

In [ ]:
if False:
  layers = [1, 2, 3, 4]

  results = {
    "layers": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for l in layers:
    results["layers"].append(l)
    
    print(f"""\n\nStarting model with {l} layers.\n\n""")
    out = train_and_eval(
      custom_layers=l,
      embed="custom",
    )

    for key in out:
      results[key].append(out[key])

  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/custom_layers_experiment.tsv", sep="\t")

## Dropout

In [ ]:
if False:
  drop = [0.000, 0.125, 0.250, 0.375, 0.500]

  results = {
    "drop_prob": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for d in drop:
    results["drop_prob"].append(d)
    
    print(f"""\n\nStarting dropout {d}.\n\n""")
    out = train_and_eval(
      custom_layers=2,
      drop_prob=d,
      embed="custom",
    )

    for key in out:
      results[key].append(out[key])

  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/custom_dropout_experiment.tsv", sep="\t")

## Weight decay

In [ ]:
if False:
  decay = [0.0, 1.0e-4, 1.0e-3, 1.0e-2, 1.0e-1]

  results = {
    "l2": [],
    "train_loss": [],
    "train_acc": [],
    "train_auc": [],
    "val_loss": [],
    "val_acc": [],
    "val_auc": []
  }

  # Run experiment.
  for l2 in decay:
    results["l2"].append(l2)
    
    print(f"""\n\nStarting weight decay {l2}.\n\n""")
    out = train_and_eval(
      custom_layers=2,
      drop_prob=0.375,
      l2=l2,
      embed="custom",
    )

    for key in out:
      results[key].append(out[key])

  # Save results.
  out = pd.DataFrame(results)
  out.to_csv("results/custom_weight_decay_experiment.tsv", sep="\t")

# Evaluation

In [ ]:
def Model() -> tf.keras.Model:
  """Final model."""
  return define_model(custom_layers=2, drop_prob=0.375,
      embed="custom", l2=0.01) 

In [ ]:
# Evaluate model.
results = model_eval(Model)

# Save results.
results["eval"].to_csv("results/final_eval.tsv", sep="\t")
np.save("pos_yhat.npy", results["pos_yhat"])
np.save("neg_yhat.npy", results["neg_yhat"])
np.save("pos_embed.npy", results["pos_embed"])
np.save("neg_embed.npy", results["neg_embed"])

## t-SNE

In [ ]:
# Embeddings.
x1 = results["pos_embed"]
x0 = results["neg_embed"]
x = np.concatenate((x1, x0), axis=0)

# Labels.
y1 = np.ones_like(results["pos_yhat"])
y0 = np.zeros_like(results["neg_yhat"])
y = np.concatenate((y1, y0), axis=0)

# Predictions.
yhat1 = results["pos_yhat"]
yhat0 = results["neg_yhat"]
yhat = np.concatenate((yhat1, yhat0), axis=0)

# Fit t-SNE.
tsne = TSNE(n_components=2)
x_tsne = tsne.fit_transform(x)

# Output.
df = pd.DataFrame(x_tsne, columns=["tsne0", "tsne1"])
df["y"] = y
df["yhat"] = yhat
df.to_csv("results/eval_tsne.tsv", sep="\t", index=False)